In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
print(tf.__version__)

/Users/sreeharirammohan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


In [2]:
X = np.load('/Volumes/Elements/Lesions/X.npy')
y = np.load('/Volumes/Elements/Lesions/y.npy')

In [2]:
#Read in images from folder
DATA_PATH = "/Volumes/Elements/Lesions"

classes = os.listdir(DATA_PATH)
del classes[0] #first entry is .DS Store not a directory

print(classes)

X = np.array([])
y = np.array([])

for class_name in classes:
    class_folder = os.path.join(DATA_PATH,class_name)

    sub_files = os.listdir(class_folder)
    
    i = 0
    
    for file in sub_files:
        
        if '.jpg' in file:
            image_filepath = os.path.join(class_folder, file)
            image_as_np_array = cv2.imread(image_filepath,flags=cv2.IMREAD_COLOR)
            
            #decrease the size of the image by 1/4 to 
            image_as_np_array = cv2.resize(image_as_np_array, (56,75), interpolation = cv2.INTER_LINEAR)
            
            #cv2.imshow('ImageWindow', image_as_np_array)
            #cv2.waitKey()
            
            #print(image_as_np_array.shape)

            X = np.append(X, image_as_np_array)
            y = np.append(y, class_name)

            print(i, end='\r', flush=True)

            i += 1
        


['Actinic_Keratosis', 'Basel_Cell_Carcinoma', 'Dermatofibroma', 'Seaborrheic Keratosis', 'X.npy', 'y.npy']


KeyboardInterrupt: 

In [3]:
X = X.reshape(-1,56,75,3)
X.shape

(1453, 56, 75, 3)

In [4]:
y.shape

(1453,)

In [5]:
#Normalize image array values between 0-1
X /= 255.0

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [9]:
print(y)

[0 0 0 ... 3 3 3]


In [10]:
np.save('/Volumes/Elements/Lesions/X.npy', X)
np.save('/Volumes/Elements/Lesions/y.npy', y)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
X_train.shape

(973, 56, 75, 3)

In [5]:
X_test.shape

(480, 56, 75, 3)

In [6]:
y_train.shape

(973,)

In [7]:
y_test.shape

(480,)

In [8]:
print(y_test)

[1 3 2 1 1 1 1 0 1 1 1 2 1 3 0 1 3 2 0 3 1 1 0 3 2 3 3 1 3 1 1 1 3 2 0 1 1
 1 3 1 0 3 0 0 3 3 0 0 1 3 0 3 1 0 0 2 0 3 3 3 3 1 0 1 1 1 2 2 1 3 3 3 0 1
 1 3 1 0 0 1 3 0 2 1 1 0 1 1 1 0 1 0 1 0 2 1 0 3 3 3 3 0 0 2 1 3 1 1 1 1 3
 1 1 2 3 3 1 1 1 1 0 1 3 1 1 3 3 1 2 1 1 3 3 1 1 1 0 3 3 3 3 0 0 2 3 0 2 3
 1 1 0 1 3 1 3 0 3 3 1 0 1 1 1 1 1 2 0 0 1 2 0 1 3 0 1 1 3 3 1 1 3 0 1 0 1
 0 0 1 1 3 1 1 3 1 1 0 0 3 2 1 3 1 2 3 1 1 1 1 0 3 0 1 3 3 3 3 1 2 0 1 1 1
 2 3 3 3 1 1 3 1 1 1 3 3 1 0 0 0 1 0 0 0 0 1 1 2 0 0 1 3 1 1 1 3 1 3 2 3 3
 1 0 1 0 3 0 3 3 0 3 2 3 3 3 1 1 1 1 3 0 3 3 1 1 1 1 1 2 3 3 2 0 3 3 1 1 1
 0 3 0 1 3 2 3 1 1 0 0 1 1 0 3 1 3 3 3 0 1 3 3 0 1 0 2 3 1 1 2 0 3 1 1 1 0
 0 0 1 3 3 2 1 3 3 0 1 3 1 1 1 3 3 1 3 3 1 2 1 0 3 1 3 3 3 3 3 1 3 0 1 3 0
 2 0 2 0 3 1 0 1 0 3 1 1 1 0 3 1 0 1 0 1 3 1 1 0 1 1 3 0 1 0 3 3 1 2 1 1 1
 0 3 3 1 2 1 0 0 1 0 0 1 0 1 3 2 1 1 0 1 1 0 3 1 1 0 1 1 3 1 3 1 1 1 0 3 3
 1 1 1 3 3 3 2 2 2 3 1 1 1 0 3 3 2 3 1 1 1 0 1 1 1 1 1 1 1 1 1 3 1 1 1 3]


In [9]:
import keras 
from keras.models import Sequential
from keras.layers import *

num_classes = 4
input_shape=(56,75,3)
epochs = 75
batch_size = 32

Using TensorFlow backend.


In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          verbose=1)

Train on 973 samples, validate on 480 samples
Epoch 1/75
973/973 [==============================] - 215s 221ms/step - loss: 1.3025 - acc: 0.3936 - val_loss: 1.3174 - val_acc: 0.4208
Epoch 2/75
768/973 [======================>.......] - ETA: 33s - loss: 1.2663 - acc: 0.4076

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    tf.keras.models.save_model(
        model,
        '/Volumes/Elements/Lesions/75_epoch_model.h5',
        overwrite=True,
        include_optimizer=True
    )

In [ ]:
converter = tf.contrib.lite.TocoConverter.from_keras_model_file('/Volumes/Elements/Lesions/model.h5')
converter.post_training_quantize = True
converter.quantize_weights = True
tflite_model = converter.convert()
open("/Volumes/Elements/Lesions/75_epoch_quantized_model.tflite", "wb").write(tflite_model)

In [ ]:
print("done")